In [1]:
import pandas as pd
import numpy as np


In [11]:
data = pd.read_csv("../data/raw/ethiopia_fi_unified_data.csv")
reference = pd.read_csv("../data/raw/reference_codes.csv")

impact_links = pd.read_csv("../data/raw/impact_links.csv")


In [12]:
data.head()
data.info()
data.columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   record_id            43 non-null     object 
 1   record_type          43 non-null     object 
 2   category             10 non-null     object 
 3   pillar               33 non-null     object 
 4   indicator            43 non-null     object 
 5   indicator_code       43 non-null     object 
 6   indicator_direction  33 non-null     object 
 7   value_numeric        33 non-null     float64
 8   value_text           10 non-null     object 
 9   value_type           43 non-null     object 
 10  unit                 33 non-null     object 
 11  observation_date     43 non-null     object 
 12  period_start         10 non-null     object 
 13  period_end           10 non-null     object 
 14  fiscal_year          43 non-null     object 
 15  gender               43 non-null     objec

Index(['record_id', 'record_type', 'category', 'pillar', 'indicator',
       'indicator_code', 'indicator_direction', 'value_numeric', 'value_text',
       'value_type', 'unit', 'observation_date', 'period_start', 'period_end',
       'fiscal_year', 'gender', 'location', 'region', 'source_name',
       'source_type', 'source_url', 'confidence', 'related_indicator',
       'relationship_type', 'impact_direction', 'impact_magnitude',
       'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country',
       'collected_by', 'collection_date', 'original_text', 'notes'],
      dtype='object')

In [14]:
# Preview the dataset
data.head()


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,REC_0001,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,22.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Baseline year,NaN
1,REC_0002,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,35.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
2,REC_0003,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,46.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
3,REC_0004,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,56.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN
4,REC_0005,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,36.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN


In [15]:
# Dataset structure
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   record_id            43 non-null     object 
 1   record_type          43 non-null     object 
 2   category             10 non-null     object 
 3   pillar               33 non-null     object 
 4   indicator            43 non-null     object 
 5   indicator_code       43 non-null     object 
 6   indicator_direction  33 non-null     object 
 7   value_numeric        33 non-null     float64
 8   value_text           10 non-null     object 
 9   value_type           43 non-null     object 
 10  unit                 33 non-null     object 
 11  observation_date     43 non-null     object 
 12  period_start         10 non-null     object 
 13  period_end           10 non-null     object 
 14  fiscal_year          43 non-null     object 
 15  gender               43 non-null     objec

In [25]:
data["record_type"].value_counts()


record_type
observation    30
event          10
target          3
Name: count, dtype: int64

In [26]:
observations = data[data["record_type"] == "observation"].copy()
events = data[data["record_type"] == "event"].copy()
targets = data[data["record_type"] == "target"].copy()
impact_links = data[data["record_type"] == "impact_link"].copy()


In [17]:
# Count by pillar (events should be NaN)
data["pillar"].value_counts(dropna=False)


pillar
ACCESS           16
USAGE            11
NaN              10
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64

In [18]:
# Distribution of confidence levels
data["confidence"].value_counts()


confidence
high      40
medium     3
Name: count, dtype: int64

In [20]:
# Check time span of observed data
observations["observation_date"].min(), observations["observation_date"].max()


('2014-12-31', '2025-12-31')

In [22]:
# Coverage per indicator
indicator_coverage = (
    observations.groupby("indicator_code")["observation_date"]
    .agg(["min", "max", "count"])
    .sort_values("count", ascending=False)
)

indicator_coverage


,min,max,count
indicator_code,,,
ACC_OWNERSHIP,2014-12-31,2024-11-29,6
ACC_FAYDA,2024-08-15,2025-05-15,3
ACC_4G_COV,2023-06-30,2025-06-30,2
ACC_MM_ACCOUNT,2021-12-31,2024-11-29,2
GEN_GAP_ACC,2021-12-31,2024-11-29,2
USG_P2P_COUNT,2024-07-07,2025-07-07,2
USG_CROSSOVER,2025-07-07,2025-07-07,1
USG_TELEBIRR_USERS,2025-06-30,2025-06-30,1
USG_P2P_VALUE,2025-07-07,2025-07-07,1


In [27]:
# View events and dates
# Events use 'indicator' as name and 'observation_date' as date
events_view = events.rename(columns={
    "indicator": "event_name",
    "observation_date": "event_date"
})

events_view[["event_name", "category", "event_date"]].sort_values("event_date")


,event_name,category,event_date
33,Telebirr Launch,product_launch,2021-05-17
41,NFIS-II Strategy Launch,policy,2021-09-01
34,Safaricom Ethiopia Commercial Launch,market_entry,2022-08-01
35,M-Pesa Ethiopia Launch,product_launch,2023-08-01
36,Fayda Digital ID Program Rollout,infrastructure,2024-01-01
37,Foreign Exchange Liberalization,policy,2024-07-29
38,P2P Transaction Count Surpasses ATM,milestone,2024-10-01
39,M-Pesa EthSwitch Integration,partnership,2025-10-27
42,Safaricom Ethiopia Price Increase,pricing,2025-12-15
40,EthioPay Instant Payment System Launch,infrastructure,2025-12-18


In [28]:
impact_links[[
    "indicator", 
    "related_indicator", 
    "impact_direction", 
    "impact_magnitude", 
    "lag_months"
]].head()


,indicator,related_indicator,impact_direction,impact_magnitude,lag_months


In [ ]:
#Add New Observation
new_obs = {
    "record_type": "observation",
    "pillar": "usage",
    "indicator": "Smartphone penetration",
    "indicator_code": "USG_SMARTPHONE_PEN",
    "value_numeric": 44.0,
    "unit": "percent",
    "observation_date": "2023-12-31",
    "source_name": "GSMA",
    "source_url": "https://www.gsma.com",
    "confidence": "medium",
    "collected_by": "Betelhem Kibret Getu",
    "collection_date": "2026-02-01",
    "original_text": "Smartphone penetration in Ethiopia reached approximately 44% in 2023",
    "notes": "Smartphone access enables digital payment usage"
}

data = pd.concat([data, pd.DataFrame([new_obs])], ignore_index=True)


In [30]:
#Add New Event
new_event = {
    "record_type": "event",
    "category": "infrastructure",
    "indicator": "4G network expansion",
    "observation_date": "2022-06-01",
    "source_name": "Ethio Telecom",
    "confidence": "medium",
    "collected_by": "Betelhem Kibret Getu",
    "collection_date": "2026-02-01",
    "notes": "Expansion of mobile broadband coverage"
}

data = pd.concat([data, pd.DataFrame([new_event])], ignore_index=True)

In [31]:
#Add impact Link
new_impact = {
    "record_type": "impact_link",
    "indicator": "4G network expansion",
    "pillar": "usage",
    "related_indicator": "USG_DIGITAL_PAYMENT",
    "impact_direction": "positive",
    "impact_magnitude": "medium",
    "lag_months": 12,
    "evidence_basis": "Improved connectivity increases digital payment adoption"
}

data = pd.concat([data, pd.DataFrame([new_impact])], ignore_index=True)


In [32]:
# Link the event to digital payment usage
new_impact_link = {
    "parent_id": "4G network expansion",
    "pillar": "usage",
    "related_indicator": "USG_DIGITAL_PAYMENT",
    "impact_direction": "positive",
    "impact_magnitude": "medium",
    "lag_months": 12,
    "evidence_basis": "Improved connectivity increases digital payment adoption"
}

impact_links = pd.concat(
    [impact_links, pd.DataFrame([new_impact_link])],
    ignore_index=True
)


In [33]:
# Save enriched datasets
data.to_csv("../data/processed/ethiopia_fi_enriched.csv", index=False)
impact_links.to_csv("../data/processed/impact_links_enriched.csv", index=False)
